In [ ]:
# https://hipolarbear.tistory.com/41 참고
# 압축된 darknet.zip 파일 압축 해제
!unzip "/content/drive/MyDrive/darknet.zip" -d "/content"

In [ ]:
# darknet이 OpenCV와 GPU를 사용할 수 있도록 Makefile을 변경 및 실행

%cd /content/darknet

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1' Makefile

!make
!chmod +x ./darknet

In [ ]:
# darknet 내의 training 데이터 검사
!rm -rf /content/darknet/backup
!ln -s /content/drive/'MyDrive'/YOLOv4-tiny2/backup /content/darknet

!sudo apt install dos2unix

!dos2unix ./data/list/train.txt
!dos2unix ./data/list/valid.txt
!dos2unix ./data/list/test.txt
!dos2unix ./data/ClassNames.names
!dos2unix ./data/hDatas.data
!dos2unix ./cfg/yolov4-tiny-custom.cfg

In [ ]:
# 생설될 이미지 확인 및 다운로드 편의를 위한 function 선언
def imShow(path): 
  import cv2 
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image, (3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# 사용할 yolo tiny 모델 다운로드

%cd /content/darknet

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [ ]:
# 학습
# time: training후 총 걸린시간 출력
# -dont_show: Loss-Window를 띄우지 않는 옵션
# -map: training시 modle의 mAP를 계산해서 보여줌과 동시에 마지막 결과값을 그래프로 저장
# tee train.log: Training시 출력되는 로그를 train.log 파일에 저장

%cd /content/darknet

!time ./darknet detector train data/hDatas.data cfg/yolov4-tiny-custom.cfg yolov4-tiny.conv.29 -dont_show -map l tee train.log

!cp -r /content/darknet/train.log /content/drive/'MyDrive'/YOLOv4-tiny2/backup/

In [ ]:
# 동영상 object detection

!./darknet detector demo data/hDatas.data cfg/yolov4-tiny-custom.cfg /content/drive/MyDrive/YOLOv4-tiny2/backup/yolov4-tiny-custom_best.weights -dont_show data/videos/box6.mp4 -thresh 0.5 -i 0 -out_filename data/test6.avi